In [0]:
dbutils.fs.rm("/FileStore/tables/users_streaming",True)
dbutils.fs.mkdirs("/FileStore/tables/users_streaming/input")
dbutils.fs.mkdirs("/FileStore/tables/users_streaming/checkpointing")

Out[5]: True

In [0]:
display(dbutils.fs.ls("/FileStore/tables/users_streaming"))

path name size modificationTime dbfs:/FileStore/tables/users_streaming/checkpointing/ checkpointing/ 0 0 dbfs:/FileStore/tables/users_streaming/input/ input/ 0 0

# Streaming - Structured Streaming

1. Read Streaming Data
2. Incremental Query
3. Write Streaming data into table

In [0]:
%sql
CREATE DATABASE usersdb;

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

custom_schema = StructType(
    [
        StructField("id", IntegerType()),
        StructField("name", StringType()),
        StructField("dob", TimestampType()),
        StructField("email", StringType()),
        StructField("gender", StringType()),
        StructField("country", StringType()),
        StructField("region", StringType()),
        StructField("city", StringType()),
        StructField("asset", IntegerType()),
        StructField("marital_status", StringType()),
    ]
)

## Read Streaming Data

In [0]:
users_streaming_df = spark.readStream.option("header","true").schema(custom_schema).format("csv").load("/FileStore/tables/users_streaming/input")

result_df = users_streaming_df.groupBy(col("country")).count()
write_data = (
  result_df.writeStream.format("delta").option("checkpointLocation","/FileStore/tables/users_streaming/checkpointing").outputMode("complete").toTable("usersdb.users_tbl")
)
write_data.awaitTermination()

## Incremental Query

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-272166417859752>:1
----> 1 result_df = users_streaming_df.groupBy(col("country")).count()

NameError: name 'users_streaming_df' is not defined